# Spark

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/13 00:37:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
numbers = sc.parallelize(range(32))

In [3]:
numbers.sum()

496

In [4]:
numbers.count()

32

In [5]:
numbers.take(4)

[0, 1, 2, 3]

In [6]:
print(numbers.collect())

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]


In [7]:
numbers.glom().collect()

[[0, 1, 2, 3, 4, 5, 6, 7],
 [8, 9, 10, 11, 12, 13, 14, 15],
 [16, 17, 18, 19, 20, 21, 22, 23],
 [24, 25, 26, 27, 28, 29, 30, 31]]

## Spark SQL

In [8]:
from pyspark.sql import SQLContext, Row

In [9]:
sparksql = SQLContext(sc)

/home/hx152/anaconda3/envs/pyspark/lib/python3.10/site-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [10]:
a = sparksql.sql('SHOW DATABASES')
a.collect()

[Row(namespace='default')]

In [11]:
a = sparksql.sql('CREATE DATABASE IF NOT EXISTS test')
a.collect()

[]

In [12]:
a = sparksql.sql('Use test')
a.collect()

[]

In [13]:
a = sparksql.sql('SHOW TABLES')
a.collect()

[]

In [14]:
a = sparksql.sql('CREATE TABLE test')
a.collect()

23/04/10 17:15:43 WARN ResolveSessionCatalog: A Hive serde table will be created as there is no table provider specified. You can set spark.sql.legacy.createHiveTableByDefault to false so that native data source table will be created instead.


AnalysisException: Hive support is required to CREATE Hive TABLE (AS SELECT);
'CreateTable `test`.`test`, org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe, ErrorIfExists


In [15]:
a = sparksql.sql('SHOW TABLES')
a.collect()

[]

# MongoDB

In [1]:
import pymongo
 
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["test"] # create a database test under the client

In [2]:
mydb.collection_names()

/tmp/ipykernel_28014/566206477.py:1: DeprecationWarning: collection_names is deprecated. Use list_collection_names instead.
  mydb.collection_names()


['testcol', 'sites']

In [3]:
mycol = mydb["testcol"] # create a collection(table) sites under the database test if not exist, use the collection if exist
mycol.drop() # clear the collection
 
x = mycol.insert_one({ "_id" : 1, "type" : "apple", "qty" : 5 }) 
x = mycol.insert_one({ "_id" : 2, "type" : "orange", "qty" : 10 })
x = mycol.insert_one({ "_id" : 3, "type" : "banana", "qty" : 15 })

In [4]:
mydict = { "_id" : 4, "type" : "potato", "qty" : 100 }
x = mycol.insert_one(mydict)

In [5]:
for x in mycol.find():
    print(x)

{'_id': 1, 'type': 'apple', 'qty': 5}
{'_id': 2, 'type': 'orange', 'qty': 10}
{'_id': 3, 'type': 'banana', 'qty': 15}
{'_id': 4, 'type': 'potato', 'qty': 100}


In [6]:
# Delete a line of the data

myquery = { "_id": 4 }
 
mycol.delete_one(myquery)

In [7]:
for x in mycol.find():
    print(x)

{'_id': 1, 'type': 'apple', 'qty': 5}
{'_id': 2, 'type': 'orange', 'qty': 10}
{'_id': 3, 'type': 'banana', 'qty': 15}


# Spark MongoDB

The link form should be   
mongodb://127.0.0.1:database.collection

In [1]:
from pyspark.sql import SparkSession
import os,sys
input_uri = "mongodb://localhost:27017/test.testcol"
output_uri = "mongodb://localhost:27017/test.testcol"
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

sparkmongodb = SparkSession \
    .builder \
    .master("spark://vm1:7077") \
    .appName("MyApp") \
    .config("spark.mongodb.read.connection.uri", input_uri) \
    .config("spark.mongodb.read.readPreference.name", "secondaryPreferred") \
    .config("spark.mongodb.write.connection.uri", output_uri) \
    .config('spark.jars.packages', 'org.mongodb.spark:mongo-spark-connector_2.12:10.1.1') \
    .getOrCreate()

:: loading settings :: url = jar:file:/usr/bigdata/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/hx152/.ivy2/cache
The jars for the packages stored in: /home/hx152/.ivy2/jars
org.mongodb.spark#mongo-spark-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-c2a07d82-1d57-4e07-a296-899d7e32ea71;1.0
	confs: [default]
	found org.mongodb.spark#mongo-spark-connector_2.12;10.1.1 in central
	found org.mongodb#mongodb-driver-sync;4.8.2 in central
	[4.8.2] org.mongodb#mongodb-driver-sync;[4.8.1,4.8.99)
	found org.mongodb#bson;4.8.2 in central
	found org.mongodb#mongodb-driver-core;4.8.2 in central
	found org.mongodb#bson-record-codec;4.8.2 in central
:: resolution report :: resolve 1012ms :: artifacts dl 8ms
	:: modules in use:
	org.mongodb#bson;4.8.2 from central in [default]
	org.mongodb#bson-record-codec;4.8.2 from central in [default]
	org.mongodb#mongodb-driver-core;4.8.2 from central in [default]
	org.mongodb#mongodb-driver-sync;4.8.2 from central in [default]
	org.mongodb.spark#mongo-spark-connector_2.

23/04/12 21:07:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
df = sparkmongodb.read.format("mongodb").load()
df.printSchema()

root
 |-- _id: integer (nullable = true)
 |-- qty: integer (nullable = true)
 |-- type: string (nullable = true)



In [4]:
df.filter(df['qty'] >= 0).show()

+---+---+------+
|_id|qty|  type|
+---+---+------+
|  1|  5| apple|
|  2| 10|orange|
|  3| 15|banana|
+---+---+------+



In [5]:
new_data = sparkmongodb.createDataFrame([("4","JULIA", 50), ("5","Gandalf", 1000)], ["_id", "type","qty"])
new_data.write.format("mongodb").mode("append").save()

23/04/12 21:11:33 WARN TaskSetManager: Lost task 7.0 in stage 2.0 (TID 9) (10.128.0.6 executor 1): com.mongodb.MongoTimeoutException: Timed out after 30000 ms while waiting to connect. Client view of cluster state is {type=UNKNOWN, servers=[{address=localhost:27017, type=UNKNOWN, state=CONNECTING, exception={com.mongodb.MongoSocketOpenException: Exception opening socket}, caused by {java.net.ConnectException: Connection refused (Connection refused)}}]
	at com.mongodb.internal.connection.BaseCluster.getDescription(BaseCluster.java:184)
	at com.mongodb.internal.connection.SingleServerCluster.getDescription(SingleServerCluster.java:46)
	at com.mongodb.client.internal.MongoClientDelegate.getConnectedClusterDescription(MongoClientDelegate.java:144)
	at com.mongodb.client.internal.MongoClientDelegate.createClientSession(MongoClientDelegate.java:101)
	at com.mongodb.client.internal.MongoClientDelegate$DelegateOperationExecutor.getClientSession(MongoClientDelegate.java:291)
	at com.mongodb.cli

23/04/12 21:12:03 WARN TaskSetManager: Lost task 7.1 in stage 2.0 (TID 10) (10.128.0.6 executor 1): com.mongodb.MongoTimeoutException: Timed out after 30000 ms while waiting to connect. Client view of cluster state is {type=UNKNOWN, servers=[{address=localhost:27017, type=UNKNOWN, state=CONNECTING, exception={com.mongodb.MongoSocketOpenException: Exception opening socket}, caused by {java.net.ConnectException: Connection refused (Connection refused)}}]
	at com.mongodb.internal.connection.BaseCluster.getDescription(BaseCluster.java:184)
	at com.mongodb.internal.connection.SingleServerCluster.getDescription(SingleServerCluster.java:46)
	at com.mongodb.client.internal.MongoClientDelegate.getConnectedClusterDescription(MongoClientDelegate.java:144)
	at com.mongodb.client.internal.MongoClientDelegate.createClientSession(MongoClientDelegate.java:101)
	at com.mongodb.client.internal.MongoClientDelegate$DelegateOperationExecutor.getClientSession(MongoClientDelegate.java:291)
	at com.mongodb.cl

In [6]:
df.filter(df['qty'] >= 0).show()

+---+----+-------+
|_id| qty|   type|
+---+----+-------+
|  1|   5|  apple|
|  2|  10| orange|
|  3|  15| banana|
|  4|  50|  JULIA|
|  5|1000|Gandalf|
+---+----+-------+

